In [25]:
import pandas as pd
import numpy as np
import time
data=pd.read_csv('input.csv',sep=',')
data.head()
machines = ['m1','m2','m3','m4','m5']
#convert into dictionary with first column as key and rest as values
data_dict = data.set_index('Run').T.to_dict('list')
jobs=data_dict
del data, data_dict

In [14]:

# # Example usage - give data here
# jobs = { # Machines: M1, M2, M3
#     'ERR016294': [9, 9, 10], #j1
#     'ERR016314': [6,10,9], #j2
#     'ERR016316': [11,7,12], #j3
# }  # Example job durations
# machines = ['m1', 'm2', 'm3']  # Example machines

##  Gredy Strategy for Plan Generation

In [37]:
def greedy_schedule(jobs, machines):
    execution_plan = {machine: [] for machine in machines}
    job_allocation = {machine: [] for machine in machines}
    J = list(jobs.keys())
    M = list(machines)
    while J: 
        for j in range(1,len(M)+1):
            shortest_job = min(J, key=lambda job: min(jobs[job]))
            print(shortest_job)
            shortest_machine = min(M, key=lambda machine: sum(execution_plan[machine]))
            print(shortest_machine)
            execution_plan[shortest_machine].append(jobs[shortest_job][machines.index(shortest_machine)])
            job_allocation[shortest_machine].append(shortest_job)
            M.remove(shortest_machine)
            J.remove(shortest_job)
            if not M:
                M = list(machines)
    return execution_plan, job_allocation
execution_plan, job_allocation = greedy_schedule(jobs, machines)
print(execution_plan)
print(job_allocation)

def longest_time(execution_plan):
    return max(sum(execution_plan[machine]) for machine in execution_plan)

print(longest_time(execution_plan))

ERR062966
m1
ERR062940
m2
ERR062953
m3
ERR019492
m4
ERR016338
m5
ERR016327
m3
SRR794387
m4
ERR022463
m1
ERR020276
m5
ERR020241
m2
{'m1': [716, 1539], 'm2': [1059, 4521], 'm3': [662, 771], 'm4': [667, 842], 'm5': [875, 3789]}
{'m1': ['ERR062966', 'ERR022463'], 'm2': ['ERR062940', 'ERR020241'], 'm3': ['ERR062953', 'ERR016327'], 'm4': ['ERR019492', 'SRR794387'], 'm5': ['ERR016338', 'ERR020276']}
5580


## Calculating Actual for Predicted

In [183]:

# # create a dataframe with duration, machine and job name for above
# df = pd.DataFrame([(job, machine, duration) for machine, durations in execution_plan.items() for job, duration in zip(job_allocation[machine], durations)], columns=['Job', 'Machine', 'Predicted Duration'])
# # print(df)actual=pd.read_csv('greedy_actual_S1.csv',sep='\t');actual.set_index("Unnamed: 0", inplace=True)
# output=df
# import warnings
# warnings.filterwarnings("ignore")
# output["Machine"].replace("m1", "1T4GPU (M0)",inplace=True)
# output["Machine"].replace("m2", "2T4GPU (M1)",inplace=True)
# output["Machine"].replace("m3", "1RTX GPU (M2)",inplace=True)
# output["Machine"].replace("m4", "2RTX GPU (M3)",inplace=True)
# output["Machine"].replace("m5", "3RTX GPU (M4)",inplace=True)
# # add a column in output 
# output["Actual Duration"] = 0
# for i in range(len(output)):
#     output["Actual Duration"][i] = actual.loc[output["Job"][i], output["Machine"][i]]
# output.columns = ["Job", "Predicted Machine", "Predicted Duration", "Actual Duration"]
# output
## group by Machine and sum Duration and Actual Duration
# grouped = df.groupby('Predicted Machine').agg({
#     'Predicted Duration': 'sum',
#     'Actual Duration': 'sum'
# }).reset_index()

# # Rename columns for clarity
# grouped.columns = ['Machine', 'Predicted (Sum)', 'Actual Duration (Sum)']

# # Display the result
# print(grouped)

## Plan Execution


In [42]:
# Input ei - Execution plan for machine mi
execution_plan, job_allocation = greedy_schedule(jobs, machines)
# for i in range(len(machines)):
#     print(f"Machine {machines[i]}: {execution_plan[machines[i]]}")
#     print(f"Jobs on Machine {machines[i]}: {job_allocation[machines[i]]}")
start= time.time() 
# pushing job on machine 
vms = ['vm1', 'vm2', 'vm3', 'vm4', 'vm5']
for i in range(len(machines)):
    # ssh vm1 touch jobs.txt
    ei = job_allocation[machines[i]]
    # ssh vm1 jobs.txt >> ei
print("copied scheduling plan on all machines")
print("Starting job execution on all machines")
for i in range(len(machines)):
    # ssh vm1 python3 script.py
    print(f"Jobs on Machine {machines[i]}: {job_allocation[machines[i]]}")
end = time.time()
#print time in seconds by round of of 5 decimal places
print("Time taken for greedy schedlule workload is: ", round(end-start,3))

copied scheduling plan on all machines
Starting job execution on all machines
Jobs on Machine m1: ['ERR018395', 'ERR018482', 'ERR018454']
Jobs on Machine m2: ['ERR016316', 'ERR062934', 'ERR023211', 'ERR020276']
Jobs on Machine m3: ['SRR799760', 'ERR018475', 'ERR018204']
Jobs on Machine m4: ['ERR016294', 'ERR062966', 'ERR019481', 'ERR020241']
Jobs on Machine m5: ['ERR062947', 'ERR020157', 'ERR022457', 'ERR020263']
Time taken for greedy schedlule workload is:  0.0


In [171]:
# SIGNAL function
# Input - operation name 'o'
o = "ERR_0_running_M1"
# Output -  Success or Failure
filename = "file-{}".format(o) # will change the path as per NFS server address
f = open(filename, "w")

# WAIT function
import time
import fcntl
import os

def wait_function(o, t):
    filename = "file-{}".format(o)
    while True:
        try:
            with open(filename, 'r+') as f:
                try:
                    fcntl.flock(f, fcntl.LOCK_EX | fcntl.LOCK_NB)
                    return "SUCCESS"
                except IOError:
                    time.sleep(10)
                    continue
        except FileNotFoundError:
            time.sleep(10)
            continue

# Example usage
o = "ERR_0_running_M1"
t = 10
result = wait_function(o, t)
print(result)

SUCCESS
